<a href="https://colab.research.google.com/github/imhyunho99/python_vision_with.Prof.Ryu/blob/main/Sivir_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Download and Learning

In [ ]:
#Roboflow version 3 dataset export

!curl -L "https://app.roboflow.com/ds/WyVRuucuYI?key=xsSw2JCXSm" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
#roboflow에서 학습시킨 데이터, 모델 Export and download

In [ ]:
#Yolov5 git commit

%cd /content
!git clone https://github.com/ultralytics/yolov5.git
#github yolo v5 clone

In [ ]:
#yolo-5 req install

%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [ ]:
#실행전에 dataset 이라는 폴더를 만들어서 train, test, data.yaml 등 yolov5 제외 모든 파일, 폴더 전부 집어 넣기기

%mkdir dataset

In [ ]:
%mv '/content/test' '/content/dataset'
%mv '/content/train' '/content/dataset'
%mv '/content/valid' '/content/dataset'
%mv '/content/README.dataset.txt' '/content/dataset'
%mv '/content/README.roboflow.txt' '/content/dataset'
%mv '/content/data.yaml' '/content/dataset'

In [ ]:
%cat /content/dataset/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['skill_success', 'skill_using']

roboflow:
  workspace: project
  project: skill_s
  version: 4
  license: CC BY 4.0
  url: https://universe.roboflow.com/project/skill_s/dataset/4

In [ ]:
#Image 싹 긁어 모으기기

%cd /
from glob import glob

img_list = glob('/content/dataset/test/images/*.jpg')
img_list += glob('/content/dataset/train/images/*.jpg')
img_list += glob('/content/dataset/valid/images/*.jpg')
print(len(img_list))

/
769


In [ ]:
#hyperParam

batch_size = 32
num_epoch = 200

In [ ]:
#끌어 모은 이미지 train, test로 split
from sklearn.model_selection import train_test_split
import torch.utils.data as data

train_img_list, test_img_list = train_test_split(img_list, test_size=0.2, random_state=42)
train_img_length, test_img_length = len(train_img_list), len(test_img_list)

train_loader =  data.DataLoader(train_img_list, batch_size=batch_size,
                            shuffle=True, drop_last = True)
test_loader = data.DataLoader(test_img_list, batch_size=batch_size,
                            shuffle=False, drop_last = True)
print(len(train_img_list), len(test_img_list))

615 154


In [ ]:
#train, test에 list up
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

In [ ]:
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/test.txt'

with open('/content/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 2, 'names': ['skill_success', 'skill_using'], 'roboflow': {'workspace': 'project', 'project': 'skill_s', 'version': 4, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/project/skill_s/dataset/4'}}
{'train': '/content/dataset/train.txt', 'val': '/content/dataset/test.txt', 'test': '../test/images', 'nc': 2, 'names': ['skill_success', 'skill_using'], 'roboflow': {'workspace': 'project', 'project': 'skill_s', 'version': 4, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/project/skill_s/dataset/4'}}


In [ ]:
%cd /content/yolov5/
#training img => 총 이미지 사이즈, batch = batch_size, epochs, data => data, 
!python train.py --batch "{batch_size}" --epochs "{num_epoch}" --data /content/dataset/data.yaml --cfg ./models/yolov5n.yaml --weights yolov5n.pt --name Sivir --optimizer Adam

/content/yolov5
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/compat/__init__.py", line 42, in tf
ImportError: cannot import name 'notf' from 'tensorboard.compat' (/usr/local/lib/python3.10/dist-packages/tensorboard/compat/__init__.py)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/yolov5/train.py", line 56, in <module>
    from utils.loggers import Loggers
  File "/content/yolov5/utils/loggers/__init__.py", line 23, in <module>
    from torch.utils.tensorboard import SummaryWriter
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py", line 12, in <module>
    from .writer import FileWriter, SummaryWriter  # noqa: F401
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/writer.py", line 16, in <module>
    from ._embedding import (
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/_embedding.

##Detection and analyse

In [ ]:
# 코드입력
!python detect.py --weights ./runs/train/Sivir/weights/best.pt --conf 0.7 --source /content/test.mp4 --save-txt --visualize=True

python3: can't open file '/content/yolov5/runs/detect/exp2/labels/yolov5/detect.py': [Errno 2] No such file or directory


In [ ]:
#Detection할 File_setting

import os
import re
import pprint

label_detected = os.chdir('/content/yolov5/runs/detect/exp2/labels/') #여기서 path 바꿔야할 수도 있음
file_list = os.listdir()
detection_set = {} 
pattern = r'\d+'
for file_name in file_list:
    file = open(file_name,'r')
    line = file.readline()
    line = line.split(' ')
    detection_class = line[0]

    numbers = re.findall(pattern, file_name)
    if numbers:
      numbers = int(numbers[0])
      detection = {numbers:detection_class}
      detection_set[numbers] = {detection_class}

detection_set = sorted(detection_set.items())
pprint.pprint(detection_set)

[(3, {'0'}),
 (4, {'0'}),
 (5, {'0'}),
 (6, {'0'}),
 (7, {'0'}),
 (8, {'0'}),
 (9, {'0'}),
 (10, {'0'}),
 (11, {'0'}),
 (12, {'0'}),
 (13, {'0'}),
 (14, {'0'}),
 (15, {'0'}),
 (16, {'0'}),
 (17, {'0'}),
 (18, {'0'}),
 (19, {'0'}),
 (20, {'0'}),
 (21, {'0'}),
 (22, {'0'}),
 (55, {'0'}),
 (57, {'0'}),
 (62, {'0'}),
 (63, {'0'}),
 (64, {'1'}),
 (65, {'1'}),
 (66, {'1'}),
 (67, {'1'}),
 (68, {'0'}),
 (69, {'0'}),
 (70, {'0'}),
 (71, {'0'}),
 (72, {'0'}),
 (73, {'0'}),
 (74, {'0'}),
 (75, {'0'}),
 (76, {'0'}),
 (77, {'0'}),
 (78, {'0'}),
 (79, {'0'}),
 (80, {'0'}),
 (81, {'0'}),
 (82, {'0'}),
 (83, {'0'}),
 (84, {'0'}),
 (85, {'0'}),
 (86, {'0'}),
 (87, {'0'}),
 (88, {'0'}),
 (89, {'0'}),
 (90, {'0'}),
 (92, {'0'}),
 (93, {'0'}),
 (99, {'0'}),
 (124, {'0'}),
 (128, {'0'}),
 (143, {'0'}),
 (144, {'0'}),
 (145, {'0'}),
 (156, {'1'}),
 (160, {'0'}),
 (161, {'0'}),
 (166, {'0'}),
 (171, {'0'}),
 (172, {'0'}),
 (173, {'0'}),
 (174, {'0'}),
 (175, {'0'}),
 (184, {'0'}),
 (187, {'0'}),
 (188, {'0'

##Ingame play

Case


1.   E스킬 사용, 아무것도 못막고 판별도 잘 한 경우

2.   E스킬 사용, 잘 막았고 판별도 잘 한 경우

3.   아무것도 못막았지만 중간에 판별이 이상하게 된 경우

4.   잘 막았지만 중간에 판별이 이상하게 된 경우

5.   처음부터 이상하게 판별한 경우
